The following cell will display the dataframe containing SAEBench releases and saes.

In [10]:
import pandas as pd
import os
import re
import json

import utils.formatting_utils as formatting_utils

overview_df = formatting_utils.make_available_sae_df(for_printing=True)

# pandas display options
max_hook_point_length = overview_df["unique_hook_points"].astype(str).map(len).max()
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", int(max_hook_point_length))

# print selected columns
show_cols = [
    "release",
    "model",
    "unique_hook_points",
    "n_saes_per_hook",
    "has_training_checkpoints",
    "saes_map",
]
overview_df[show_cols]

,release,model,unique_hook_points,n_saes_per_hook,has_training_checkpoints,saes_map
0,sae_bench_gemma-2-2b_sweep_standard_ctx128_ef2_0824,gemma-2-2b,"[blocks.15.hook_resid_post, blocks.3.hook_resid_post, blocks.7.hook_resid_post, blocks.11.hook_resid_post, blocks.19.hook_resid_post]",6,True,{'blocks.3.hook_resid_post__trainer_1_step_29292': 'gemma-2-2b_sweep_standard_ctx128_ef2_0824/resid_post_layer_3_checkpoints/trainer_1_step...
1,sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8_0824,gemma-2-2b,"[blocks.15.hook_resid_post, blocks.3.hook_resid_post, blocks.7.hook_resid_post, blocks.11.hook_resid_post, blocks.19.hook_resid_post]",6,True,{'blocks.3.hook_resid_post__trainer_5_step_4882': 'gemma-2-2b_sweep_standard_ctx128_ef8_0824/resid_post_layer_3_checkpoints/trainer_5_step_...
2,sae_bench_gemma-2-2b_sweep_topk_ctx128_ef2_0824,gemma-2-2b,"[blocks.15.hook_resid_post, blocks.3.hook_resid_post, blocks.7.hook_resid_post, blocks.11.hook_resid_post, blocks.19.hook_resid_post]",6,True,{'blocks.3.hook_resid_post__trainer_2_step_9764': 'gemma-2-2b_sweep_topk_ctx128_ef2_0824/resid_post_layer_3_checkpoints/trainer_2_step_9764...
3,sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824,gemma-2-2b,"[blocks.15.hook_resid_post, blocks.3.hook_resid_post, blocks.7.hook_resid_post, blocks.11.hook_resid_post, blocks.19.hook_resid_post]",6,True,"{'blocks.3.hook_resid_post__trainer_3': 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_3/trainer_3', 'blocks.3.hook_resid_post__tr..."
4,sae_bench_pythia70m_sweep_gated_ctx128_0730,pythia-70m,"[blocks.4.hook_resid_post, blocks.3.hook_resid_post]",20,False,"{'blocks.3.hook_resid_post__trainer_0': 'pythia70m_sweep_gated_ctx128_0730/resid_post_layer_3/trainer_0', 'blocks.3.hook_resid_post__traine..."
5,sae_bench_pythia70m_sweep_panneal_ctx128_0730,pythia-70m,"[blocks.4.hook_resid_post, blocks.3.hook_resid_post]",28,False,"{'blocks.3.hook_resid_post__trainer_16': 'pythia70m_sweep_panneal_ctx128_0730/resid_post_layer_3/trainer_16', 'blocks.3.hook_resid_post__tr..."
6,sae_bench_pythia70m_sweep_standard_ctx128_0712,pythia-70m,"[blocks.4.hook_resid_post, blocks.3.hook_resid_post]",24,False,"{'blocks.3.hook_resid_post__trainer_10': 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_3/trainer_10', 'blocks.3.hook_resid_post__t..."
7,sae_bench_pythia70m_sweep_topk_ctx128_0730,pythia-70m,"[blocks.4.hook_resid_post, blocks.3.hook_resid_post]",24,False,"{'blocks.3.hook_resid_post__trainer_0': 'pythia70m_sweep_topk_ctx128_0730/resid_post_layer_3/trainer_0', 'blocks.3.hook_resid_post__trainer..."


Each row is a "release" which has multiple SAEs which may have different configs / match different hook points in a model. These are 8 SAE Bench releases: 4 for Pythia and 4 for Gemma-2-2B.

Each release will contain an saes_map, a dict of sae_id: sae_name. The `sae_ids` are SAE Lens specific, used to load the SAEs into SAELens.

In this project, we use the `sae_names` as keys in our results dictionaries, rather than the sae_ids. This is because the names are unique, and there's no possibility of mixing data between different SAEs.

In [11]:
sae_df = formatting_utils.make_available_sae_df(for_printing=False)

sae_release = "sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824"

sae_id_to_name_map = sae_df.saes_map[sae_release]
sae_name_to_id_map = {v: k for k, v in sae_id_to_name_map.items()}

print(f"First sae id: {list(sae_id_to_name_map.keys())[0]}")
print(f"First sae name: {list(sae_id_to_name_map.values())[0]}")

First sae id: blocks.3.hook_resid_post__trainer_3
First sae name: gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_3/trainer_3


As an example, here's a dictionary of sae_release: all saes for a given layer for that Gemma release. This is the input format that we are using for `sparse_probing/`. Note that in this particular example we are not including checkpoints.

In [12]:
sae_releases = [
    "gemma-scope-2b-pt-res",
    "sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824",
    "sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8_0824",
]

layer = 19

selected_saes_dict = {}

for release in sae_releases:
    if "gemma-scope" in release:
        selected_saes_dict[release] = formatting_utils.find_gemmascope_average_l0_sae_names(layer)
    else:
        selected_saes_dict[release] = formatting_utils.filter_sae_names(
            sae_names=release, layers=[layer], include_checkpoints=False, trainer_ids=None
        )

for key in selected_saes_dict:
    print("\n\n", key, "\n\n",selected_saes_dict[key])



 gemma-scope-2b-pt-res 

 ['layer_19/width_16k/average_l0_137', 'layer_19/width_16k/average_l0_23', 'layer_19/width_16k/average_l0_279', 'layer_19/width_16k/average_l0_40', 'layer_19/width_16k/average_l0_73']


 sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824 

 ['gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19/trainer_5', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19/trainer_4', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19/trainer_3', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19/trainer_2', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19/trainer_1', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19/trainer_0']


 sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8_0824 

 ['gemma-2-2b_sweep_standard_ctx128_ef8_0824/resid_post_layer_19/trainer_2', 'gemma-2-2b_sweep_standard_ctx128_ef8_0824/resid_post_layer_19/trainer_3', 'gemma-2-2b_sweep_standard_ctx128_ef8_0824/resid_post_layer_19/trainer_4', 'gemma-2-2b_sweep_standa

This cells gets all Gemma checkpoints. Notice that it also includes the final SAE, which is not included in the checkpoints folder.

In [13]:
sae_releases = [
    # "gemma-scope-2b-pt-res",
    "sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824",
    "sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8_0824",
]

layer = 19

selected_saes_dict = {}

for release in sae_releases:
    if "gemma-scope" in release:
        selected_saes_dict[release] = formatting_utils.find_gemmascope_average_l0_sae_names(layer)
    else:
        selected_saes_dict[release] = formatting_utils.filter_sae_names(
            sae_names=release, layers=[layer], include_checkpoints=True, trainer_ids=None
        )

for key in selected_saes_dict:
    print("\n\n", key, "\n\n",selected_saes_dict[key])



 sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824 

 ['gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_4_step_0', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_4882', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_488', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_48', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_15440', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_1544', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_154', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_3_step_0', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_0_step_154', 'gemma-2-2b_sweep_topk_ctx128_ef8_0824/resid_post_layer_19_checkpoints/trainer_0_step_0', 'gemma-2-2b_sweep_topk_ctx12

This cell gets all standard and topk SAEs for Pythia layer 4. 

In [14]:
pythia_sae_releases = [
    "sae_bench_pythia70m_sweep_standard_ctx128_0712",
    "sae_bench_pythia70m_sweep_topk_ctx128_0730",
    # "sae_bench_pythia70m_sweep_gated_ctx128_0730",
    # "sae_bench_pythia70m_sweep_panneal_ctx128_0730",
]

layer = 4

selected_saes_dict = {}

for release in pythia_sae_releases:
    if "gemma-scope" in release:
        selected_saes_dict[release] = formatting_utils.find_gemmascope_average_l0_sae_names(layer)
    else:
        selected_saes_dict[release] = formatting_utils.filter_sae_names(
            sae_names=release, layers=[layer], include_checkpoints=False, trainer_ids=None
        )

for key in selected_saes_dict:
    print("\n\n", key, "\n\n",selected_saes_dict[key])



 sae_bench_pythia70m_sweep_standard_ctx128_0712 

 ['pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_0', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_11', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_10', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_1', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_13', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_14', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_16', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_17', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_18', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_2', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_20', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_21', 'pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_12', 'pythia70m_sweep_standard_ctx128_0712/r

When testing we may want to run only a single SAE. This cell only runs a single Pythia TopK SAE.

In [15]:
pythia_sae_releases = [
    # "sae_bench_pythia70m_sweep_standard_ctx128_0712",
    "sae_bench_pythia70m_sweep_topk_ctx128_0730",
    # "sae_bench_pythia70m_sweep_gated_ctx128_0730",
    # "sae_bench_pythia70m_sweep_panneal_ctx128_0730",
]

layer = 4

selected_saes_dict = {}

for release in pythia_sae_releases:
    if "gemma-scope" in release:
        selected_saes_dict[release] = formatting_utils.find_gemmascope_average_l0_sae_names(layer)
    else:
        selected_saes_dict[release] = formatting_utils.filter_sae_names(
            sae_names=release, layers=[layer], include_checkpoints=False, trainer_ids=[10]
        )

for key in selected_saes_dict:
    print(key, selected_saes_dict[key])

sae_bench_pythia70m_sweep_topk_ctx128_0730 ['pythia70m_sweep_topk_ctx128_0730/resid_post_layer_4/trainer_10']


Here is an example of loading a Pythia SAE.

In [16]:
from sae_lens import SAE
from sae_lens.sae import TopK

pythia_sae_release = "sae_bench_pythia70m_sweep_topk_ctx128_0730"

sae_name = 'pythia70m_sweep_topk_ctx128_0730/resid_post_layer_4/trainer_10'

sae_id_to_name_map = sae_df.saes_map[pythia_sae_release]
sae_name_to_id_map = {v: k for k, v in sae_id_to_name_map.items()}

sae_id = sae_name_to_id_map[sae_name]

device = "cpu"
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release=pythia_sae_release,
    sae_id=sae_id,
    device=device,
)
sae = sae.to(device=device)

print(f"Is sae topk? {isinstance(sae.activation_fn, TopK)}")

assert isinstance(sae.activation_fn, TopK), "This sae is not a topk sae, you probably have an old sae_lens version"

print(cfg_dict)

/opt/conda/envs/saebench/lib/python3.11/site-packages/sae_lens/toolkit/pretrained_sae_loaders.py:480: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder = torch.load(enco

Is sae topk? True
{'architecture': 'standard', 'd_in': 512, 'd_sae': 16384, 'dtype': 'float32', 'device': 'cpu', 'model_name': 'pythia-70m-deduped', 'hook_name': 'blocks.4.hook_resid_post', 'hook_layer': 4, 'hook_head_index': None, 'activation_fn_str': 'topk', 'activation_fn_kwargs': {'k': 80}, 'apply_b_dec_to_input': True, 'finetuning_scaling_factor': False, 'sae_lens_training_version': None, 'prepend_bos': True, 'dataset_path': 'monology/pile-uncopyrighted', 'dataset_trust_remote_code': False, 'context_size': 128, 'normalize_activations': 'none', 'neuronpedia_id': None}


In [17]:
sae.cfg

SAEConfig(architecture='standard', d_in=512, d_sae=16384, activation_fn_str='topk', apply_b_dec_to_input=True, finetuning_scaling_factor=False, context_size=128, model_name='pythia-70m-deduped', hook_name='blocks.4.hook_resid_post', hook_layer=4, hook_head_index=None, prepend_bos=True, dataset_path='monology/pile-uncopyrighted', dataset_trust_remote_code=False, normalize_activations='none', dtype='float32', device='cpu', sae_lens_training_version=None, activation_fn_kwargs={'k': 80}, neuronpedia_id='pythia-70m/4-sae_bench-topk-res-16k-trainer_10_step_final', model_from_pretrained_kwargs={})

This repo already contains info on all SAEs we're using at  `sae_bench_data/{release_name}_data.json`. This contains the config used in the `dictionary_learning` repo, which includes training hyperparameters, SAE type, etc. It also contains the `basic_eval_results`, which includes the `l0` and `frac_recovered`, which was obtained using the `dictionary_learning evaluate()` function. These are already computed, so we can use them when making graphs.

In [ ]:
release_name = "sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824"
sae_data_filename = f"sae_bench_data/{release_name}_data.json"

with open(sae_data_filename, "r") as f:
    sae_data = json.load(f)

As we can see, `sae_data` contains two keys: 'sae_config_dictionary_learning' and 'basic_eval_results'. Within each key, we have all SAE names for that release.

In [ ]:
example_sae_names = list(sae_data["sae_config_dictionary_learning"].keys())[:5]
example_sae_name = example_sae_names[0]
example_sae_config = sae_data["sae_config_dictionary_learning"][example_sae_name]
example_basic_eval_result = sae_data["basic_eval_results"][example_sae_name]

print(sae_data.keys())
print("\nExample evaluated SAEs:\n", example_sae_names)
print("\nFirst SAE config:\n", example_sae_config)
print("\nFirst basic eval result:\n", example_basic_eval_result)

The results file of the custom eval you're implementing will contain a `custom_eval_config` and `custom_eval_results`. 

`custom_eval_config` contains a dict of hyperparameters and config values to reproduce the results.

`custom_eval_results` contains a dict, where every key is an SAE name, and every value is another dict containing various results from the eval. This dict can be immediately loaded in to `graph_sae_results.ipynb` to create various plots.

In [ ]:
folder_path = "evals/sparse_probing/results"
filename = "example_gemma-2-2b_layer_19_eval_results.json"
filepath = os.path.join(folder_path, filename)

with open(filepath, "r") as f:
    custom_eval_results = json.load(f)

print(custom_eval_results.keys())
print(f'\nCustom eval config:\n{custom_eval_results["custom_eval_config"]}')
print(f'\nCustom eval results for {example_sae_name}:\n{custom_eval_results["custom_eval_results"][example_sae_name]}')